# Assignment 5 - MCMC Methods in Julia
## Pranshu Gaur - 200703

### Section 2 - Barker's method and the two-coin algorithm

Given a current state of the Markov chain $x$, and a proposal density $q(x, y)$, the Barker's Acceptance Probability is 
$$\alpha_{B}(x, y) = \frac{\pi(y)q(y, x)}{\pi(x)q(x, y) + \pi(y)q(y, x)}$$

We can obtain at time m+1 realization by drawing $U \sim U[0,1]$ and checking if $U \leq \alpha_{B}(x, y)$ but this is not possible when $\alpha_{B}(x, y)$ cannot be evaluated, so the way out is to construct a Bernoulli factory to obtain events of probability $\alpha_{B}(x, y)$ without evaluating it.

The Algorithm for the same is : 

#### Algorithm 1 : Baker's MCMC for $x_{m+1}$
* Draw $y \sim q(x_m, dy)$ 
* Draw $A \sim$ Bern $(\alpha_B(x_m, y))$
* If A = 1 then
    * $x_{m+1} = y$
* If A = 0 then
    * $x_{m+1} = x_m$



Goncalves proposed a different algorithm to sample events with probability $\alpha_B(x,y)$ \
He supposed $\pi(x)q(x, y) = c_xp_x$, where $c_x$ is the local bound on $\pi(x)q(x, y)$ such that $p_x$ remains a valid probability. \
That is, $c_x \geq \pi(x)q(x, y)$ and hence $p_x = \pi(x)q(x, y) c_x^{-1}$

The Algorithm is as follows :

#### Algorithm 2 : two-coin algorithm for $\alpha_B(x_m, y)$
* Draw $C1 \sim (\large{\frac{c_y}{c_y + c_x}})$ 
* If $C1 = 1$ then
    * Draw $C2 \sim$ Bern $(p_y)$
        * If $C2 = 1$ then
            output 1
        * If $C2 = 0$ then
            go to Step 1    
* If $C1 = 0$ then
    * Draw $C2 \sim$ Bern $(p_x)$
        * If $C2 = 1$ then
            output 0
        * If $C2 = 0$ then
            go to Step 1
<!-- Draw C1 ∼ Bern
cy
cy + cx

2: if C1 = 1 then
3: Draw C2 ∼ Bern(py)
4: if C2 = 1 then
5: output 1
6: if C2 = 0 then
7: go to Step 1
8: if C1 = 0 then
9: Draw C2 ∼ Bern(px)
10: if C2 = 1 then
11: output 0
12: if C2 = 0 then
13: go to Step 1 -->



As proved in Assignment 2, the two-coin Algorithm generates a $Bern(\large{\frac{p_yc_y}{p_xc_x + p_yc_y}})$ event.

As proved in Assignment 2, the average number of iterations to get an output is, $\large{N = \frac{1}{P_{out}} = \frac{c_x + c_y}{p_xc_x + p_yc_y}}$, hence the execution time depends heavily on the upper bounds, $c_x$ and $c_y$. If the bounds are loose then the algorithm yields a larger mean execution time.
 


### Section 3 - Portkey Barker's method

The main inefficiency in Barker's method is the inefficiency of two-coin algorithm, hence let us consider the acceptance probability for a proposed density $q(x,y)$ as 
$$ \alpha_B(x,y) = \frac{\pi(y)q(y, x)}{\pi(x)q(x, y) + \pi(y)q(y, x) + d(x,y)}$$

here $d(x,y) = d(y,x) \geq 0 $ 

This $\alpha_B(x,y) $ yields a $\pi$ reversible Markov Chain which means that $π(y)q(y, x)\alpha(y, x) = π(x)q(x, y)\alpha(x, y).$

For a user-chosen $0 < \beta ≤ 1$, consider the following Portkey Barker’s Acceptance
Probability: $$\alpha_{(\beta)}(x, y) := \frac{π(y)q(y, x)} {π(y)q(y, x) + π(x)q(x, y) + \frac{(1 − \beta)}{\beta}(cx + cy)}$$

where $\beta \sim 1$

The portkey algorithm modifies the two-coin algorithm by introducing a new step in the beginning where proposals are rejected with a probability of $1 - \beta$. The Algorithm is as follows :

#### Algorithm 3: Portkey two-coin algorithm

* Draw $S \sim$ Bern $(\beta)$ 
* If $S = 0$ then
    output 0
* If $S = 1$ then
    * Draw $C1 \sim (\large{\frac{c_y}{c_y + c_x}})$ 
    * If $C1 = 1$ then
        * Draw $C2 \sim$ Bern $(p_y)$
            * If $C2 = 1$ then
                 output 1
            * If $C2 = 0$ then
                 go to Step 1    
    * If $C1 = 0$ then
        * Draw $C2 \sim$ Bern $(p_x)$
            * If $C2 = 1$ then
                 output 0
            * If $C2 = 0$ then
                 go to Step 1

Using the result for Algorithm 2, the number of loops until the algorithm stops is given by, $s_{\beta} = (1 - \beta) + \beta.\large{\frac{c_yp_y + c_xp_x}{c_x + c_y}}$

When $\beta = 1$, the Portkey Algorithm is same as Barker's two-coin Algorithm. Comparing the ratio of mean execution time of both the algorithms, 
$$\frac{1/s_1}{1/s_{\beta}} = \frac{s_{\beta}}{s_1} = (1 - \beta).(\frac{c_yp_y + c_xp_x}{c_x + c_y})^{-1}$$
Thus, if $\frac{c_yp_y + c_xp_x}{c_x + c_y}$ tends to 0, then the original two-coin algorithm is not useful as the execution time ratio diverges to infinity, on the other hand if the value tends to 1, then both the algorithms are equally efficient. When $p_x$ or $p_y$ is small, it is advised to take large value of $\beta$.

### Section 4 - Flipped portkey two-coin algorithm

A 